In [1]:
import pickle
import os
import numpy as np

project_folder = "."

with open(os.path.join(project_folder,"data","train_test.pkl"), "rb") as f:
    X_train, X_test, y_train, y_test = pickle.load(f)

In [2]:
type(X_test)

scipy.sparse.coo.coo_matrix

In [3]:
X_train_np = X_train.toarray()
X_test_np = (X_test.toarray()) 


y_train_np = np.array(y_train)
y_test_np  = np.array(y_test)

In [4]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
np_data_scaled = sc.fit_transform(X_train_np)
np_data_scaled_test = sc.transform(X_test_np)

In [5]:
### Imports 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.utils import normalize, to_categorical
from sklearn.metrics import roc_curve, auc
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import initializers
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import to_categorical

In [6]:
# Converting Y variables to Categorical

y_train_categorical = to_categorical(y_train_np)
y_test_categorical = to_categorical(y_test_np)

In [7]:
model = Sequential()

model.add(tf.keras.Input(shape=(68,)))

model.add(Dense(500))
model.add(LeakyReLU())
model.add(Dropout(rate=0.3))

model.add(Dense(300))
model.add(LeakyReLU())
model.add(Dropout(rate=0.1))

model.add(Dense(100))
model.add(LeakyReLU())

model.add(Dense(2,activation = 'sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               34500     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 500)               0         
_________________________________________________________________
dropout (Dropout)            (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               150300    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               3

In [8]:
# Compiling the ANN

earlyStopping = EarlyStopping(monitor='val_auc', mode='max', min_delta = 0.0001 , patience = 10,restore_best_weights=True)

callbacks_a = [earlyStopping]

adam = optimizers.Adam(lr = 0.0001)

model.compile(optimizer = optimizers.Adam(), loss = 'binary_crossentropy', metrics = [tf.keras.metrics.AUC(),tf.keras.metrics.Recall()])

In [9]:
# Fitting the ANN
model.fit(np_data_scaled, y_train_categorical, batch_size = 256, validation_data=(np_data_scaled_test,y_test_categorical), epochs = 1000 , callbacks = callbacks_a)

Train on 128000 samples, validate on 32000 samples
Epoch 1/1000
128000/128000 [==============================] - 10s 78us/sample - loss: 0.3453 - auc: 0.9276 - recall: 0.8476 - val_loss: 0.2498 - val_auc: 0.9622 - val_recall: 0.9003
Epoch 2/1000
128000/128000 [==============================] - 9s 71us/sample - loss: 0.2343 - auc: 0.9673 - recall: 0.9074 - val_loss: 0.1959 - val_auc: 0.9764 - val_recall: 0.9231
Epoch 3/1000
128000/128000 [==============================] - 9s 70us/sample - loss: 0.1988 - auc: 0.9761 - recall: 0.9241 - val_loss: 0.1790 - val_auc: 0.9802 - val_recall: 0.9323
Epoch 4/1000
128000/128000 [==============================] - 9s 70us/sample - loss: 0.1783 - auc: 0.9804 - recall: 0.9343 - val_loss: 0.1614 - val_auc: 0.9836 - val_recall: 0.9410
Epoch 5/1000
128000/128000 [==============================] - 9s 71us/sample - loss: 0.1683 - auc: 0.9825 - recall: 0.9391 - val_loss: 0.1465 - val_auc: 0.9863 - val_recall: 0.9476
Epoch 6/1000
128000/128000 [===============

In [10]:
test = np_data_scaled_test[1]

In [11]:
test.shape

(68,)

In [12]:
model.predict(np_data_scaled_test)

array([[9.9897516e-01, 1.0997653e-03],
       [6.1494410e-03, 9.9467844e-01],
       [5.6259429e-01, 4.0306184e-01],
       ...,
       [9.9405563e-01, 5.0358474e-03],
       [9.9765551e-01, 4.1653216e-03],
       [9.9942595e-01, 4.7898293e-04]], dtype=float32)

In [13]:
from sklearn import metrics
import numpy as np
from tqdm import tqdm

probas = list(np.arange(0.01, 0.4, 0.01))

def calcCost(model, X, y, proba, fp = 10, fn = 500):
    y_hat_proba = model.predict(X,use_multiprocessing=True)
    y_hat = (y_hat_proba[:,1] > proba).astype(int)
    mt = metrics.confusion_matrix(y, y_hat)
    acc = metrics.accuracy_score(y, y_hat)
    precision = metrics.precision_score(y, y_hat)
    return {
        "probability":proba,
        "accuracy": acc,
        "precision": precision,
        "fn":mt[1,0],
        "fn_cost":mt[1,0]*fn,
        "fp":mt[0,1],
        "fp_cost":mt[0,1]*fp
    }

cost = [ calcCost(model, np_data_scaled_test, y_test, proba) for proba in tqdm(probas) ]

100%|██████████| 39/39 [00:57<00:00,  1.47s/it]


In [14]:
import pandas as pd

cost_df = pd.DataFrame(cost)

cost_df["total_cost"] = cost_df.fn_cost + cost_df.fp_cost

cost_df.sort_values(by=["total_cost"], ascending=True).head()

,probability,accuracy,precision,fn,fn_cost,fp,fp_cost,total_cost
3,0.04,0.899344,0.802114,95,47500,3126,31260,78760
4,0.05,0.908531,0.817695,104,52000,2823,28230,80230
2,0.03,0.885813,0.780576,91,45500,3563,35630,81130
5,0.06,0.915531,0.829999,111,55500,2592,25920,81420
1,0.02,0.863094,0.746923,84,42000,4297,42970,84970


In [16]:
model.save('models/model_5.h5')